# Benchmark of pyfoamalgo

Author: Jun Zhu, August 23, 2020

In [ ]:
import random
import numpy as np

import pyfoamalgo
print(pyfoamalgo.__version__)

from pyfoamalgo import mask_image_data, nanmean_image_data, nansum

In [ ]:
import multiprocessing as mp

mp.cpu_count()

Assume we want to process a train of detector images.

In [ ]:
images = np.random.randn(128, 1200, 1200).astype(np.float32)
images[:, ::2, ::2] = np.nan

### Benchmark nanmean

In [ ]:
%timeit np.nanmean(images, axis=0)

In [ ]:
%timeit nanmean_image_data(images)

A common use case is to apply `nanmean` to a list of selected images.

In [ ]:
# select 120 images out of 128 ones
selected = random.sample(range(images.shape[0]), 120)

In [ ]:
# numpy is slower than operating on all the images since it copies the data when 'selected' is a list.
%timeit np.nanmean(images[selected], axis=0)

In [ ]:
%timeit nanmean_image_data(images, kept=selected)

### Benchmark masking

threshold mask

In [ ]:
imgs = images.copy()
%timeit imgs[(imgs > 1) | (imgs < -1)] = np.nan

In [ ]:
imgs = images.copy()
%timeit mask_image_data(imgs, threshold_mask=(-1, 1))

image mask

In [ ]:
image_mask = np.ones((1200, 1200), dtype=bool)
image_mask[::3, ::3] = np.nan

In [ ]:
imgs = images.copy()
%timeit imgs[:, image_mask] = np.nan

In [ ]:
imgs = images.copy()
%timeit mask_image_data(imgs, image_mask=image_mask)

threshold mask and image mask

In [ ]:
imgs = images.copy()
%timeit imgs[(image_mask) | (imgs > 1) | (imgs < -1)] = np.nan

In [ ]:
imgs = images.copy()
%timeit mask_image_data(imgs, image_mask=image_mask, threshold_mask=(-1, 1))

### Benchmark nan-statistics

Non-paralleled funtion can be also much faster than `numpy`.

In [ ]:
images = np.random.randn(1200, 1200).astype(np.float32)
images[::2, ::2] = np.nan

In [ ]:
%timeit np.nansum(images)

In [ ]:
%timeit nansum(images)